In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
# Read in the Dataframe
dfuse=pd.read_csv("TweetsDF.csv")
dftest = dfuse.head(100)
dftest

,Unnamed: 0,tweetID,text,hashtags,hashtag_count,url_count,favorite_count,retweet_count,followers_count,swear_count,emoji_count,emoticon_count
0,0,6.65776E+17,"As campaign focus turns to foreign policy, Cli...",NaN,0,1,1,0,5949,0,0,0
1,1,6.65776E+17,Hillary Clinton's 10 Most Hawkish Moments From...,NaN,0,1,0,0,78,0,0,0
2,2,6.65776E+17,Watch The Full Second Democratic Debate Here -...,NaN,0,1,0,0,156,0,0,0
3,3,6.65776E+17,"Clinton plays defense on Iraq, Wall Street at ...",[empresa],1,1,0,0,725,0,0,0
4,4,6.65776E+17,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,478,0,0,0
5,5,6.65776E+17,#CNN Hillary Clinton at Democratic debate: ISI...,[CNN],1,1,0,0,1587,0,0,0
6,6,6.65776E+17,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,1894,0,0,0
7,7,6.65776E+17,Both Hillary and Ronda got beat up tonight. Ba...,"[DemDebate, UFC193]",2,0,6,2,486,0,0,0
8,8,6.65776E+17,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,364,0,0,0
9,9,6.65776E+17,Democratic Candidates Respond to Paris Attacks...,"[demdebate, breakingnews, India]",3,1,0,0,1894,0,0,0


In [3]:
practice = dfuse.text[0]
practice

'As campaign focus turns to foreign policy, Clinton points to her experience - Washington Post https://t.co/zZPfrjntpN'

In [4]:
# this is the python wrapper for Sentiment analysis
import labMTsimple.storyLab as sl

In [5]:
# Import the dictionary
lang = 'english'
labMT,labMTvector,labMTwordList = sl.emotionFileReader(stopval=0.0,lang=lang,returnVector=True)

In [6]:
# create the vectors with the scores and the stop words
practiceValence,practiceFvec = sl.emotion(practice,labMT,shift=True,happsList=labMTvector)
practiceValence
practiceStoppedVec = sl.stopper(practiceFvec,labMTvector,labMTwordList,stopVal=1.0)
finalValence = sl.emotionV(practiceStoppedVec,labMTvector)

In [7]:
# testing to see the words taken from a tweet
for i in range(0, len(practiceFvec)):
    if practiceFvec[i] == 1:
        print labMTwordList[i]

experience
focus
points
washington
her
clinton
foreign
post
campaign
as
turns
policy


In [8]:
varlisttest = list()
for i in range(0, len(practiceStoppedVec)):
    if practiceStoppedVec[i] == 1:
        varlisttest.append(labMTvector[i])

In [9]:
# more testing with the practice tweet
for i in range(0, len(practiceStoppedVec)):
    if practiceStoppedVec[i] == 1:
        print labMTwordList[i]

experience
focus
points
washington


In [10]:
import labMTsimple.storyLab as sl

# importing the dictionary
lang = 'english'
labMT,labMTvector,labMTwordList = sl.emotionFileReader(stopval=0.0,lang=lang,returnVector=True)

sentiment_list = list()
variance_list = list()

for i in range(0, len(dfuse)):
    tweettext = str(dfuse.text[i])
    # calculate sentiment score for each tweet
    tweetValence,tweetFvec = sl.emotion(tweettext,labMT,shift=True,happsList=labMTvector)
    
    # set score for neutral words to 0 and generate real sentiment score
    tweetStoppedVec = sl.stopper(tweetFvec,labMTvector,labMTwordList,stopVal=1.0)
    finaltweetsent = sl.emotionV(tweetStoppedVec,labMTvector)
    
    # handle corner case for when all words are neutral. emotionV automatically sets it to -1 when words are all neutral
    # we will set to 5 because it is in the middle of our 1 to 9 happiness scale
    if finaltweetsent == -1:
        finaltweetsent = 5
    
    # calculate variance of sentiment of words in each tweet
    varlist = list()
    for i in range(0, len(tweetStoppedVec)):
        if tweetStoppedVec[i] == 1:
            varlist.append(labMTvector[i])
    tweetvariance = np.var(varlist)
    # handle corner case for tweet with no sentiment score
    if np.isnan(tweetvariance):
        tweetvariance = 0
    
    sentiment_list.append(finaltweetsent)
    variance_list.append(tweetvariance)    
    
# store values in dataframe
dfuse['sentiment'] = sentiment_list
dfuse['sentiment_var'] = variance_list
    
    
    
    
    

/Users/Sarah/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [11]:
dfuse = dfuse.drop('Unnamed: 0', 1)

In [12]:
# Identify text of tweet that is splitting the line in Excel: found to be due to the "\r" character
dfuse.text[77]

"Me-speaks about the democratic debate &amp; why Bernie needs to win\rPpl - you are only in high school you don't know what you're talking about"

In [13]:
# remove the \r characters that are creating the unwanted line breaks
for i in range(0, len(dfuse)):
    remover = str(dfuse.text[i])
    if '\r' in remover:
        newstr = remover.replace("\r", "")
        dfuse.text[i] = newstr
        

/Users/Sarah/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
# save the dataframe to a csv
dfuse.to_csv("finalsentiment.csv")

In [15]:
# make sure the dataframe loads properly
dftest2=pd.read_csv("finalsentiment.csv")

In [16]:
dftest2.head()

,Unnamed: 0,tweetID,text,hashtags,hashtag_count,url_count,favorite_count,retweet_count,followers_count,swear_count,emoji_count,emoticon_count,sentiment,sentiment_var
0,0,6.65776E+17,"As campaign focus turns to foreign policy, Cli...",NaN,0,1,1,0,5949,0,0,0,6.135000,0.015075
1,1,6.65776E+17,Hillary Clinton's 10 Most Hawkish Moments From...,NaN,0,1,0,0,78,0,0,0,6.213333,0.008089
2,2,6.65776E+17,Watch The Full Second Democratic Debate Here -...,NaN,0,1,0,0,156,0,0,0,6.500000,0.320000
3,3,6.65776E+17,"Clinton plays defense on Iraq, Wall Street at ...",[empresa],1,1,0,0,725,0,0,0,5.646667,1.552089
4,4,6.65776E+17,Democratic Candidates Respond to Paris Attacks...,"[demdebate, Nigeria, Ghana, USA]",4,1,0,0,478,0,0,0,5.475000,3.084675
